In [ ]:
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2

# Charger un modèle YOLOv8 pré-entraîné
model = YOLO('yolov8n.pt')

# Fonction pour afficher les résultats visuellement
def plot_results(results, img_path):
    img = cv2.imread(img_path)
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            cls = box.cls[0]
            label = f'{model.names[int(cls)]} {conf:.2f}'

            # Définir la couleur en fonction de la confiance
            if conf > 0.9:
                color = (0, 0, 255)  # Rouge pour confiance 90-100%
            elif conf > 0.75:
                color = (238, 130, 238)  # Violet pour confiance 75-90%
            else:
                color = (0, 255, 0)  # Vert pour le reste
            
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            print(f'Label: {model.names[int(cls)]}, Probabilité: {conf*100:.2f}%')
    
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Parcourir toutes les images dans le dossier ./img
image_folder = './img'
for img_file in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_file)
    if os.path.isfile(img_path) and img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Exécuter une inférence sur l'image
        results = model.predict(img_path)
        # Afficher les résultats dans la console
        print(results)
        # Afficher les résultats visuellement
        plot_results(results, img_path)


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from playsound import playsound
import threading
import time

# Charger un modèle YOLOv8 pré-entraîné
model = YOLO('yolov8n.pt')

# Fonction pour calculer la couleur en fonction de la confiance
def get_color(conf):
    # Du vert (0, 255, 0) au rouge (0, 0, 255)
    red = int(min(255, 255 * (1 - conf)))
    green = int(min(255, 255 * conf))
    return (0, green, red)

# Fonction pour jouer le son dans un thread séparé
def play_sound():
    playsound('alert.wav')

# Fonction pour arrêter le son
def stop_sound():
    playsound('silence.wav')

# Fonction pour afficher les résultats visuellement
def plot_results(img, results, person_detected):
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            cls = box.cls[0]
            label = f'{model.names[int(cls)]} {conf*100:.2f}%'

            color = get_color(conf)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Afficher le message si une personne est détectée avec une probabilité > 80%
            if model.names[int(cls)] == 'person' and conf * 100 > 80:
                person_detected = True
                cv2.putText(img, 'Personne detectee', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    return person_detected

# Fonction pour gérer le son d'alerte
def alert_sound():
    while True:
        if person_detected_event.is_set() and not alarm_disabled:
            threading.Thread(target=play_sound).start()
            time.sleep(2)  # Le son joue pendant 2 secondes
            if person_detected_event.is_set():
                threading.Thread(target=stop_sound).start()
                time.sleep(3)  # Pause de 3 secondes avant de pouvoir rejouer le son
        else:
            time.sleep(1)

# Initialiser la capture vidéo depuis la caméra
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Erreur: Impossible d'ouvrir la caméra.")
    exit()

# Fonction pour gérer les modes
mode_detection = False
alarm_disabled = False
person_detected_event = threading.Event()
threading.Thread(target=alert_sound, daemon=True).start()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Erreur: Impossible de lire le cadre.")
        break

    # Vérifier le mode actuel et exécuter les actions appropriées
    if mode_detection:
        # Exécuter une inférence sur le cadre
        results = model.predict(frame)
        # Afficher les résultats visuellement sur le cadre
        person_detected = plot_results(frame, results, False)
        if person_detected:
            person_detected_event.set()
        else:
            person_detected_event.clear()

    # Afficher le cadre avec les prédictions (ou normal)
    cv2.imshow('YOLOv8 Camera', frame)

    # Gestion des entrées clavier
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('d'):
        mode_detection = not mode_detection
    elif key == ord('a'):
        alarm_disabled = not alarm_disabled

# Libérer la capture vidéo et fermer toutes les fenêtres
cap.release()
cv2.destroyAllWindows()



    Error 275 for command:
        open silence.wav
    Impossible de trouver le fichier spécifié. Assurez-vous que le chemin d’accès et le nom du fichier sont corrects.

    Error 263 for command:
        close silence.wav
    Le périphérique spécifié n’est pas ouvert ou n’est pas reconnu par MCI.
Failed to close the file: silence.wav
Exception in thread Thread-43 (stop_sound):
Traceback (most recent call last):
  File "c:\Users\thoma\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\thoma\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\thoma\AppData\Local\Temp\ipykernel_33512\4007914535.py", line 24, in stop_sound
  File "c:\Users\thoma\AppData\Local\Programs\Python\Python311\Lib\site-packages\playsound.py", line 72, in _playsoundWin
    winCommand(u'open {}'.format(sound))
  File "c:\Users\thoma\AppData\Local\Programs\Pyt